In [11]:
import numpy as np
from ipywidgets import Button, HBox, VBox, Output
from IPython.display import display
from ipycanvas import Canvas


class Connect4:
    def __init__(self):
        self.rows = 6
        self.cols = 7
        self.board = np.zeros((self.rows, self.cols), dtype=int)
        self.current_player = 1
        self.game_over = False
        self.output = Output()
        
        # High-resolution canvas (for sharp drawing)
        self.canvas = Canvas(width=700, height=600)
        
        # Smaller DISPLAY size (scaled down visually)
        self.canvas.layout = {'width': '350px', 'height': '300px'}  # Fix: Use layout
        
        # Buttons sized to match columns
        self.buttons = [Button(description=str(i), layout={'width': '50px'}) for i in range(self.cols)]
        self.setup_gui()

    def setup_gui(self):
        self.canvas.fill_style = 'blue'
        self.canvas.fill_rect(0, 0, 700, 600)
        self.draw_board()
        
        for i, button in enumerate(self.buttons):
            button.on_click(self.make_move_handler(i))
        
        display(VBox([self.canvas, HBox(self.buttons), self.output]))

    def draw_board(self):
        self.canvas.clear()
        self.canvas.fill_style = 'blue'
        self.canvas.fill_rect(0, 0, 700, 600)
        
        cell_size = 100  # 700/7 = 100, 600/6 = 100 (high-res grid)
        radius = 40
        
        # Draw pieces from BOTTOM UP
        for row in range(self.rows):
            for col in range(self.cols):
                x = col * cell_size + cell_size // 2
                y = (self.rows - 1 - row) * cell_size + cell_size // 2  # Flip vertically
                
                if self.board[row][col] == 1:
                    self.canvas.fill_style = 'red'
                elif self.board[row][col] == 2:
                    self.canvas.fill_style = 'yellow'
                else:
                    self.canvas.fill_style = 'white'
                
                self.canvas.begin_path()
                self.canvas.arc(x, y, radius, 0, 2 * np.pi)
                self.canvas.fill()

    
    def make_move_handler(self, col):
        def handler(button):
            if not self.game_over and self.current_player == 1 and self.is_valid_move(col):
                self.make_move(col, 1)
                self.draw_board()
                if self.check_winner() == 1:
                    self.end_game("Player 1 wins!")
                elif self.is_board_full():
                    self.end_game("It's a draw!")
                else:
                    self.current_player = 2
                    self.ai_move()
        return handler

    def ai_move(self):
        valid_moves = [col for col in range(self.cols) if self.is_valid_move(col)]
        col = np.random.choice(valid_moves)
        self.make_move(col, 2)
        self.draw_board()
        if self.check_winner() == 2:
            self.end_game("Player 2 wins!")
        elif self.is_board_full():
            self.end_game("It's a draw!")
        else:
            self.current_player = 1

    def end_game(self, message):
        self.game_over = True
        self.output.clear_output()
        with self.output:
            print(message)

    def is_valid_move(self, col):
        return self.board[self.rows - 1][col] == 0

    def make_move(self, col, player):
        for row in range(self.rows):
            if self.board[row][col] == 0:
                self.board[row][col] = player
                return True
        return False

    def check_winner(self):
        for player in [1, 2]:
            # Horizontal
            for row in range(self.rows):
                for col in range(self.cols - 3):
                    if all(self.board[row][col + i] == player for i in range(4)):
                        return player
            # Vertical
            for col in range(self.cols):
                for row in range(self.rows - 3):
                    if all(self.board[row + i][col] == player for i in range(4)):
                        return player
            # Diagonal (positive slope)
            for row in range(self.rows - 3):
                for col in range(self.cols - 3):
                    if all(self.board[row + i][col + i] == player for i in range(4)):
                        return player
            # Diagonal (negative slope)
            for row in range(3, self.rows):
                for col in range(self.cols - 3):
                    if all(self.board[row - i][col + i] == player for i in range(4)):
                        return player
        return 0

    def is_board_full(self):
        return all(self.board[self.rows - 1][col] != 0 for col in range(self.cols))

if __name__ == "__main__":
    game = Connect4()